## __Text mining y Procesamiento de Lenguaje Natural (NLP)__

__Profesor__: Anthony D. Cho

__Tema__: Clustering de documentos

__Método__: K-Means

***

__Dependencias__

```{python}
    python -m pip install nltk spacy
    python -m spacy download en_core_web_sm
    python -m spacy download es_core_news_sm
```

## Librerias

In [ ]:
import re
from glob import glob
import matplotlib.pyplot as plt
from pandas import DataFrame
from tqdm import tqdm

from string import punctuation
from spacy.lang.es.stop_words import STOP_WORDS
from spacy import load

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_distances

## Instancia del modelo de lenguaje
nlp = load('es_core_news_sm')

## Carga de documentos

In [ ]:
## Encontrar la ruta de cada archivo de interes
path_docs = glob('*/doc*.txt')

## Almacenamiendo de contenido de los documentos e id (nombre del archivo)
corpus, doc_id = [], [] 

## Incio de proceso de carga de documentos
if len(path_docs):
    for file in path_docs:

        ## Se carga el texto
        text = open(file, 'r', encoding='utf-8').read()
        
        ## Se almacena el texto
        corpus.append(text)
        
        id = file.split('\\')[-1].split('.')[0]

        ## Se almacena el id
        doc_id.append(id)
else:
    print('No corpus have found.')

In [ ]:
doc_id

#### Preprocesamiento

In [ ]:
## Limpieza de textos
cleanTexts = []

for doc in corpus:

    # ## Remover numeros y puntuaciones
    doc = re.sub(r'[\"\¿\°\d+]', '', doc)
    doc = [s for s in doc if s not in punctuation]
    doc = ''.join(doc)

    ## Normalización y remover stopwords
    documento = nlp(doc.lower())
    tokens = [word.text for word in documento]
    doc = [word for word in tokens if word not in STOP_WORDS]
    doc = ' '.join(doc)
    doc = re.sub(pattern='\s+', repl=' ', string=doc)
    
    ## Aplicar lemmatización
    documento = nlp(doc)
    lemmas = [word.lemma_ for word in documento]
    doc = ' '.join(lemmas)
    doc = re.sub(pattern='\s+', repl=' ', string=doc)

    ## Almacenado de contenido procesado
    cleanTexts.append(doc)

## Mostar contenido procesado
cleanTexts
    

In [ ]:
## Instancia del modelo
model = TfidfVectorizer(use_idf=True)

## Ajuste del modelo y retorno de TF matrix como (docs, term)
tf_sparse = model.fit_transform(cleanTexts)

## Extraer la lista de los palabras que representan las columnas de la matriz generada
vocabulary = model.get_feature_names_out()

print('(shape) TFxIDF (terms, docs): {}'.format(tf_sparse.shape))

#### Clustering

In [ ]:
## Clustering 
n_clusters = 3

## Instancia del modelo
model = KMeans(n_clusters=n_clusters,
               n_init=10,  
               random_state=0)

## Ajuste del modelo
model.fit(tf_sparse)

## Extracción de las etiquetas asignadas
etiquetas = model.labels_
etiquetas

#### Visualización en 2D

In [ ]:
## Se calcula la distancia basada en la distancia del coseno
distance = cosine_distances(tf_sparse)

## Instancia del modelo Multidimensional scaling (MDS)
mds = MDS(n_components=2, dissimilarity='precomputed',
          normalized_stress='auto',
          random_state=20231020, n_jobs=-1)

## Aplicamos el método MDS a la data
coordinates = mds.fit_transform(distance)
X, Y = coordinates[:, 0], coordinates[:, 1]

## Cluster-docs display
plt.figure(figsize=(8, 8))
for i in set(etiquetas):
    
    ## Docs filtering by label
    mask = (etiquetas == i)
    
    ## Extract coordinates
    temp_X, temp_Y = X[mask], Y[mask]
    
    ## plot coordinates
    plt.plot(temp_X, temp_Y, 'o', label=f'Cluster {i}')

## show legend
plt.legend(loc=[1.01, 0.5])

## add names to display space
for i, doc_name in enumerate(doc_id):
    plt.text(X[i], Y[i], doc_name, size=8)

## complements
plt.xlabel('latent feature 1'); plt.ylabel('latent feature 2');
plt.title('Cluster visualization')
plt.tight_layout()

## Busqueda del mejor número de clusteres

In [ ]:
## Almacenador de resultados
scores = {'k': [], 
          'SSE': [],
          'Silhouette': [],
          'Davies_Bouldin': []}

for n_clusters in tqdm(range(2, 7)):

    ## Instancia del modelo
    model = KMeans(n_clusters=n_clusters,
                   n_init=10,  
                   random_state=0)

    ## Ajuste del modelo
    model.fit(tf_sparse)

    ## computo de la métrica de Silhountte
    scores['Silhouette'].append(silhouette_score(tf_sparse.toarray(), model.labels_))

    ## computo de la métrica de Davis Bouldin
    scores['Davies_Bouldin'].append(davies_bouldin_score(tf_sparse.toarray(), model.labels_))

    ## Almacenado del SSE y número de clusteres
    scores['SSE'].append(model.inertia_)
    scores['k'].append(n_clusters)


## Convertir los resultados en un dataframe
scores = DataFrame(scores)

In [ ]:
scores

In [ ]:
plt.figure(figsize=(10, 8))
for i in range(1, 4):
    plt.subplot(3, 1, i)
    plt.plot(scores.iloc[:,0], scores.iloc[:,i], '-o', label=scores.columns[i])
    plt.title(scores.columns[i])
    plt.xlabel('Cantidad de clusteres')
plt.tight_layout()
plt.show()